# Recreating the baseline scenario for DACCS

In [ ]:
import ixmp
import message_ix
import numpy as np
import pandas as pd
import yaml

from collections.abc import Mapping
from itertools import repeat
from message_ix.models import MESSAGE_ITEMS
from message_ix.utils import make_df
from message_ix.tools.add_dac import *

%matplotlib inline


mp = ixmp.Platform()


## Clone baseline and create emission bound scenario

In [ ]:
# this is to import scenario from data base and clone to local
base = message_ix.Scenario(mp, model='GENIE_SSP2_v4.1.7', scenario="EN_NPi2020-DACm-MP-median-stor3-phs_1000f")
scen = base.clone(
    "GENIE_sandbox",
    "baselineYWP",
    "baseline scenario to reproduce feasibility paper",
    keep_solution=False, 
)
scen.check_out()

year_df = scen.vintage_and_active_years()
vintage_years, act_years = year_df["year_vtg"], year_df["year_act"]
model_horizon = scen.set("year")


In [ ]:
# list of parameters to remove
params = ['inv_cost','fix_cost','input','capacity_factor','technical_lifetime',
          'construction_time','abs_cost_activity_soft_up','growth_activity_lo',
          'level_cost_activity_soft_lo','level_cost_activity_soft_up','relation_activity',
          'var_cost','output','emission_factor','soft_activity_lo','soft_activity_up',
          'growth_activity_up','initial_activity_up']
# list of technologies to remove
techs = ['dac_2_exports', 'dac_1_exports', 'dac_3_exports', 
         'dac_4_exports', 'dac_5_exports', 'dacco2_tr_dis', 'DAC_mpen']
# removing parameters
for par in params:
    scen.remove_par(par, scen.par(par,{'technology':techs}))

# removing daccs from technology set
scen.remove_set('technology',techs)


In [ ]:
# Removing emissions bound
scen.remove_par('bound_emission', scen.par('bound_emission',{'node':'World'}))


## Run the model

In [ ]:
scen.commit(comment="Feasibility paper baseline copy with daccs removed")
scen.set_as_default()
scen.solve(solve_options={'barcrossalg': '2','scaind':'1'}) # solve_options={'barcrossalg': '2','scaind':'1'}
scen.var("OBJ")["lvl"]

## Close connection

In [ ]:
mp.close_db()